In [116]:
import geopandas as gpd
import pandas as pd
import itertools

In [281]:
vic_pop = pd.read_csv('../../data/curated/VicPopSA2.csv')
income = pd.read_csv('../../data/landing/income.csv')
sa2Areas = pd.read_csv('../../data/landing/SA2_2021_AUST.csv')

all_properties = pd.read_csv('../../data/landing/Historical Property Data/AllProperties.csv')
flat1 = pd.read_csv('../../data/landing/Historical Property Data/1_Flat.csv')
flat2 = pd.read_csv('../../data/landing/Historical Property Data/2_Flat.csv')
house2 = pd.read_csv('../../data/landing/Historical Property Data/2_House.csv')
house3 = pd.read_csv('../../data/landing/Historical Property Data/3_House.csv')
house4 = pd.read_csv('../../data/landing/Historical Property Data/4_House.csv')

/tmp/ipykernel_32598/3812216178.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  income = pd.read_csv('../../data/landing/income.csv')


In [259]:
# Getting rid of unneccessary rows in sa2Areas

sa2Areas = sa2Areas['SA2_CODE_2021,SA2_NAME_2021,STATE_NAME_2021'.split(',')]

sa2Areas = sa2Areas[sa2Areas['STATE_NAME_2021'] == 'Victoria']

# Convert data types
sa2Areas['SA2_CODE_2021'] = sa2Areas['SA2_CODE_2021'].astype(int)
sa2Areas['SA2_NAME_2021'] = sa2Areas['SA2_NAME_2021'].astype(str)
sa2Areas['STATE_NAME_2021'] = sa2Areas['STATE_NAME_2021'].astype(str)

sa2_code_range = (sa2Areas['SA2_CODE_2021'].min(), sa2Areas['SA2_CODE_2021'].max())


In [260]:
# Filter out non-numeric values in the 'Code' column of the income DataFrame
income = income[pd.to_numeric(income['Code'], errors='coerce').notnull()]
income['Code'] = income['Code'].astype(int)
income = income[income['Code'].between(sa2_code_range[0], sa2_code_range[1])]


income_columns = income.columns.tolist()
columns_to_keep = income_columns[0:3]+income_columns[-9:-1]
income_new = income[columns_to_keep]
income_new.rename(columns={'Label': 'Suburb'})
# MASSIVE DATA LOSS

# Regex
income_new = income_new.replace('-',None)
income_new = income_new.replace('\,', '', regex=True)

# Convert Data Types
float_columns = ['Median equivalised total household income (weekly) ($)','$1-$499 per week (%).1','$500-$999 per week (%).1',
                 '$1000-$1999 per week (%).1','$2000-$2999 per week (%).1','$3000 or more per week (%).1','Nil income (%).1','Partial income stated (%)']
int_columns = ['Year','Code']

for col in float_columns:
    income_new[col] = income_new[col].astype(float)
for col in int_columns:
    income_new[col] = income_new[col].astype(int)

In [261]:
vic_pop = vic_pop[vic_pop['SA2 code'].between(sa2_code_range[0], sa2_code_range[1])]


int_columns = 'SA2 code,2023 no,2022-23 no,Natural increase,Net internal migration,Net overseas migration'.strip().split(',')


for col in vic_pop.columns:
    if str(col) in int_columns:
        vic_pop[col] = vic_pop[col].astype(int)
    elif str(col) == 'SA2 name':
        vic_pop[col] = vic_pop[col].astype(str)
    else:
        vic_pop[col] = vic_pop[col].astype(float)

vic_pop.dtypes

SA2 code                    int64
SA2 name                   object
2023 no                     int64
2022-23 no                  int64
2022-23 %                 float64
Natural increase            int64
Net internal migration      int64
Net overseas migration      int64
Area (km2)                float64
persons/km2               float64
dtype: object

In [285]:
property_data = [all_properties, flat1, flat2, house2, house3, house4]
months = ['Mar', 'Jun', 'Sep', 'Dec']
years = range(2000,2024)


for df in property_data:

    # Rename columns and drop unnecessary rows
    df.rename(columns={'Unnamed: 0': 'Suburb'}, inplace=True)
    months_cycle = itertools.cycle(months)

    i = 1
    year_index = 0
    while i < len(df.columns)-1:
        month = str(next(months_cycle))

        df.rename(columns={df.columns[i]: f'{years[year_index]} {month} Count'}, inplace=True)
        df.rename(columns={df.columns[i+1]: f'{years[year_index]} {month} Median'}, inplace=True)

        year_index = year_index + 1 if month == 'Dec' else year_index
        i += 2
        
    df.drop(df[df['Suburb'] == 'Group Total'].index, inplace=True)

    # Regex
    df.replace('\,', '', regex=True,inplace=True)
    df.replace('-',0,inplace=True)
    df.replace({'\$': ''}, regex=True,inplace=True)
    df.fillna(0,inplace=True)

    # Convert Data Types
    for i in range(len(df.columns)):
        if i == 0:
            df[df.columns[i]] = df[df.columns[i]].astype(str)
        else:
            df[df.columns[i]] = df[df.columns[i]].astype(int)


In [288]:
# Save all the new data
income_new.to_csv('../../data/curated/income_by_suburb.csv', index=False)
vic_pop.to_csv('../../data/curated/VicPopSA2.csv', index=False)
sa2Areas.to_csv('../../data/curated/SA2_Victoria.csv', index=False)
all_properties.to_csv('../../data/curated/AllProperties.csv', index=False)
flat1.to_csv('../../data/curated/1_Flat.csv', index=False)
flat2.to_csv('../../data/curated/2_Flat.csv', index=False)
house2.to_csv('../../data/curated/2_House.csv', index=False)
house3.to_csv('../../data/curated/3_House.csv', index=False)
house4.to_csv('../../data/curated/4_House.csv', index = False)